In [1]:
# LIBRARIES
# *********
import requests # request for the web pagegg
from fake_useragent import UserAgent # to fake meta data
from requests.packages.urllib3.exceptions import InsecureRequestWarning # handle while requesting for the web page
from selenium import webdriver # create a simulation of the pseudo-user
import time # set sleep on web page on every click, handling nginx error
import datetime  # record start time
from bs4 import BeautifulSoup # to get web page soure
import pandas as pd # create df
import numpy as np # manipulate with shape of the df
import pyodbc
import csv # get rid off quotes
import json
lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]

In [15]:
baseColNames  = ['Область', 'ДатаПодачиЗаявки', 'Заявитель',
                 'Поставщик', 'Продукт',
                 'Объем', 'БазоваяЕдиницаИзмерения',
                 'ПлощадьИспользованияГа', 'Статус', 'ТипЗаявки',
                 'ДатаПринятияЗаявки', 'ДатаОтклонения',
                 'СуммаСубсидийТг', 'ПричинаОтклонения',
                 'ПримечаниеКОтрицательномуРешению', 'group', 'year']
      

#---------------------------------------------------------------------------------
baseColNamesReorderForFerti = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Производитель удобрений', 'Наименование удобрения', 
                        'Объем удобрения', 'Единица измерения',
                        'Площадь использования, га', 'Статус', 'Тип заявки',
                        'Дата принятия заявки', 'Дата отрицательного решения', 
                        'Сумма субсидий (расчетная), тг.', 'Причина отрицательного решения',
                        'Примечание к отрицательному решению', 'group', 'year']

#---------------------------------------------------------------------------------
baseColNamesReorderForSeeds = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Поставщик семян', 'Наименование', 'Объем семян в базовой ед.измерения',
                        'Базовая единица измерения', 'Площадь использования, га', 'Статус',
                        'Тип заявки', 'Дата принятия заявки', 'Дата отклонения',
                        'Сумма субсидий (расчетная), тг.', 'Причина отклонения',
                        'Примечание к отрицательному решению', 'group']


#------------------------------------ P E S T I C I D E S ---------------------------------------
p20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2020.csv', encoding='utf-8', sep='|')
p20['group'] = "Пестициды"
p20 = p20.iloc[:, 2:]
p20['year'] = "2020"
# print(p20.columns)
print(len(p20.columns))


p19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2019.csv', encoding='utf-8', sep='|')
p19['group'] = "Пестициды"
p19 = p19.iloc[:, 2:]
p19['year'] = "2019"
# print(p19.columns)
print(len(p19.columns))


p = pd.concat([p19, p20], ignore_index = True)
p.columns = baseColNames

#------------------------------------ F E R T L I Z E R S ---------------------------------------

f20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2020.csv', encoding='utf-8', sep='|')
f20['group'] = "Удобрения"
f20 = f20.iloc[:, 2:]
f20['year'] = "2020"

f19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2019.csv', encoding='utf-8', sep='|')
f19['group'] = "Удобрения"
f19 = f19.iloc[:, 2:]
f19['year'] = "2019"

f = pd.concat([f19, f20], ignore_index = True)
f = f[baseColNamesReorderForFerti]
f.columns = baseColNames

#------------------------------------ S E E D S ------------------------------------------------
sAll =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Семена2019_2020.csv', encoding='utf-8', sep='|')
sAll['group'] = "Семена"
sAll = sAll.iloc[:, 1:] #check columns
sAll = sAll[baseColNamesReorderForSeeds]

s19 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2019", case = False)]
s19['year'] = "2019"
s20 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2020", case = False)]
s20['year'] = "2020"

s = pd.concat([s19, s20], ignore_index = True)
s.columns = baseColNames
#------------------------------------ A L L -- C A T E G O R I E S ------------------------------------------------
q = pd.concat([p, f, s], ignore_index = True)
q['applicantID'] =  q[['Заявитель', 'Область']].agg('-'.join, axis=1)
q = q.replace(r'\n',' ', regex=True)
q = q.replace(r'~',' ', regex=True)


#------------------------------------ C O N S T R A C T I O N S ------------------------------------------------
colsToUpper = [2,3]
for colUpper in range(len(colsToUpper)):
    q.iloc[:, colsToUpper[colUpper]] = q.iloc[:, colsToUpper[colUpper]].str.upper()
q = q.replace(to_replace = ["ТОВАРИЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ", "ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ",
                            'ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ'], 
              value = "ТОО",
              regex=True)
q['Заявитель'] = q['Заявитель'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['Поставщик'] = q['Поставщик'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['Область'] = q['Область'].replace("Шымкент г.а.",  "г.Шымкент", regex=True)
q['Область'] = q['Область'].replace("Алматы г.а.",  "г.Алматы", regex=True)
q['Область'] = q['Область'].replace("Нур-Султан г.а.",  "г.Нур-Султан", regex=True)

#------------------------------------ D I S T R I B U T O R S ------------------------------------------------
pak = ['ТОО ПОЛГАР АГРОКЕМ', 'ТОО ПЕРВАЯ АГРОХИМИЧЕСКАЯ КОМПАНИЯ']
q['Поставщик'].replace({pak[0]: 'ПАК', pak[1]: 'ПАК', 'ТОО ALEM AGRO TRADE': 'ALEM AGRO'}, inplace=True, regex=True)
q['Поставщик'] = q['Поставщик'].str.replace('ТОО ALEM AGRO HOLDING АЛЕМАГРО ХОЛДИНГ', 'ALEM AGRO')

#------------------------------------  A A   l o g i c a l  c o l u m n ------------------------------------------------- 
q['АА'] = np.where(q['Поставщик'] == 'ALEM AGRO', 'Да', 'Нет')
aaClient = q[['АА', 'applicantID']].loc[q['АА'] == "Да"].drop_duplicates().reset_index().iloc[:,2:]
q['isSubsAaClient'] = q.applicantID.isin(aaClient.applicantID).astype(bool)

#------------------------------------ P A T H  ------------------------------------
q.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\q.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
#------------------------------------ R A N D O M   5   R O W S ------------------------------------
q.sample(5)

17
17


<ipython-input-15-b67fa2e47eb4>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s19['year'] = "2019"
<ipython-input-15-b67fa2e47eb4>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s20['year'] = "2020"


TypeError: sequence item 0: expected str instance, float found

In [17]:
# q.sample(3)

,Область,ДатаПодачиЗаявки,Заявитель,Поставщик,Продукт,Объем,БазоваяЕдиницаИзмерения,ПлощадьИспользованияГа,Статус,ТипЗаявки,ДатаПринятияЗаявки,ДатаОтклонения,СуммаСубсидийТг,ПричинаОтклонения,ПримечаниеКОтрицательномуРешению,group,year
164697,Туркестанская область,26.08.2020 15:36,ИЛЬЯСОВ ЖУМАБАЙ САЙЛАУОВИЧ,"КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ""ОРАЗГУЛЬ""",МАКТААРАЛ 4011,"260,00",кг,"9,45",Заявка оплачена,Переводная заявка об оплате причитающихся субс...,02.09.2020 20:42,NaN,"45 500,00",NaN,NaN,Семена,2020
149962,Туркестанская область,23.04.2020 03:49,ЖУНИСОВА БАЗАРКУЛ НУРМАНОВНА,"Акционерное общество ""КазАзот""","Селитра аммиачная марки Б (N 34,4)","3,85",тонн,"9,55",Отозвана,Переводная,NaN,NaN,"140 525,00",NaN,NaN,Удобрения,2020
144080,Костанайская область,14.05.2020 20:55,"КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ""Павловский А.Э""","Товарищество с ограниченной ответственностью ""...",Жидкое комплексное минеральное удобрение «Волс...,"300,00",литр,"380,00",Заявка оплачена,Прямая,16.05.2020 16:12,NaN,"59 945,00",NaN,NaN,Удобрения,2020


In [24]:
# q[np.isnan(q['Область'])]
nans = q[q['Заявитель'].isnull()]
# uniques = pd.unique(nans[['group', 'year']].values.ravel())
nans.drop_duplicates()

,Область,ДатаПодачиЗаявки,Заявитель,Поставщик,Продукт,Объем,БазоваяЕдиницаИзмерения,ПлощадьИспользованияГа,Статус,ТипЗаявки,ДатаПринятияЗаявки,ДатаОтклонения,СуммаСубсидийТг,ПричинаОтклонения,ПримечаниеКОтрицательномуРешению,group,year
396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пестициды,2019
41274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Пестициды,2020
107544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Удобрения,2019
129022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Удобрения,2020
